# Deep Learning: CNN: Cancer Detection
**Thomas Bohn**   --   **2025-09-14**

{{xxxxx}}  

--  [Main Report](xxxx)  --  [Github Repo](xxxx)  --  [Presentation Slides](xxx)  --  [Presentation Video](xxx) --  

# 1.&nbsp;Introduction

**Problem Statement**

{{xxxxx}}

**Why is it Important?**

{{xxxxx}}

**Limitations of Existing Solutions**

{{xxxxx}}

**Contribution**

{{xxxxx}}

**DataSet**

{{xxxxx}}


## Python Libraries

The following python libraries are used in this notebook.

In [22]:
# File system manangement
import time, datetime, psutil, os
import shutil
import zipfile
from google.colab import userdata
from google.colab import files

# Data manipulation
import numpy as np
import pandas as pd
import math

# Install text storage and manipulation
import re
import json
import pickle
import textwrap

#Install Image processing
from PIL import Image

##################################

# Plotting and visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
import seaborn as sns
sns.set_theme()

# Train-test split and cross validation
from sklearn.model_selection import train_test_split, ParameterGrid

# Model Evaluation
from sklearn import metrics
from sklearn.metrics import accuracy_score

#################################

# Import Tensor Flow and Keras
import tensorflow as tf
import keras
from keras import layers, models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

## Global Variables

The following are global variables referenced in this notebook.

In [2]:
# Recording the starting time, complemented with a stopping time check in the end to compute process runtime
start = time.time()

# Class representing the OS process and having memory_info() method to compute process memory usage
process = psutil.Process(os.getpid())

In [28]:
# Global Debug flag used to turn on and off more chatty blocks of code
gDEBUG = True
if gDEBUG: print('Debug is set to:', gDEBUG)
# Global Level of Detail of table stats and details
gLOD = 2
print('Level of Detail for functions is set to:', gLOD)

Debug is set to: True
Level of Detail for functions is set to: 2


# 2.&nbsp;Data Source

In this section, the code loads the dataset from Google Drive.

{{xxxxx}}

## Import the Data

In [23]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

target_file = 'histopathologic-cancer-detection.zip'
source_path_root =  '/content/drive/MyDrive/[1.4] MsDS Class Files/-- DTSA 5511 Deep Learning/data'
destination_path_root = '/content'

shutil.copy(source_path_root+'/'+target_file, destination_path_root+'/')

print(os.listdir(destination_path_root+'/'))

Mounted at /content/drive
['.config', 'histopathologic-cancer-detection.zip', 'drive', 'sample_data']


In [25]:
# Unzip the files
zip_file_path = destination_path_root+'/'+target_file

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents into the specified folder
        zip_ref.extractall(destination_path_root)

In [26]:
file_path_root =  '/content'


file_path_train   = file_path_root + '/train'
file_path_test    = file_path_root + '/test'
file_path_labels  = file_path_root + '/train_labels.csv'
file_path_sample  = file_path_root + '/sample_submission.csv'

# Load train labels from CSV file
train_labels = pd.read_csv(file_path_labels)

# Load test labels from CSV file
sample_output = pd.read_csv(file_path_sample)

In [29]:
if gDEBUG:
  # Preview the Data Sources
  print("Sample of Training Labels")
  display(train_labels.head(5))
  print()
  print("Sample of Results Output:")
  display(sample_output.head(5))

Sample of Training Labels


,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0



Sample of Results Output:


,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0
1,95596b92e5066c5c52466c90b69ff089b39f2737,0
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0
3,2c35657e312966e9294eac6841726ff3a748febf,0
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0


In [32]:
# Get training and testing files
train_file_names = os.listdir(file_path_train)
test_file_names = os.listdir(file_path_test)

print('Total Training Files: ', len(train_file_names))
print('Total Testing Files: ', len(test_file_names))
pct_assigned_train = round(len(train_file_names) / (len(train_file_names) + len(test_file_names)), 3)
print('Percent Assigned to Training: ', pct_assigned_train)
print()

print('Preview of training image file names: ')
print(train_file_names[:10])
print('Preview of testing image file names: ')
print(test_file_names[:10])

Total Training Files:  220025
Total Testing Files:  57458
Percent Assigned to Training:  0.793

Preview of training image file names: 
['c823338229d6b73b00b7b4e55e7b699a3401ee3f.tif', '8d82f3bd017724fb7647df92fd501328d382a707.tif', 'dd3213361e77be50c160617223ca076111d3569f.tif', 'bef0b669b473bc2aff91467822f31167acaaf609.tif', '8dc08b595911a645f67bc3a5d6048cca2ed09985.tif', 'b9617df1085ba874cbbed39345d2c7ff09bcee5b.tif', '1ec5d65d222384d8e1fde325080a37dcb3bdf8f2.tif', '81b6880e7c30834bb6921fa925f7b5424c6a85da.tif', 'bef553a4d5fb9b60c3f938bfde475e9b194c7351.tif', 'bcc86858083649cf21f87e8a81af73585246be4f.tif']
Preview of testing image file names: 
['77c8a993a6554adb01847c6dad00debbdac42e6c.tif', '55244dff457bea22745ae67b3c4a87ac3c260ac6.tif', '50d0c5068db837b20292564c903ed1f902246d61.tif', 'e607d2e3693253a1ac55324de79ebe1f685451eb.tif', '2b29fe81d4545bc0a443c679a17eeeb8dcbe0483.tif', 'af438608ba4cb92d35cae44ed710f143563977cb.tif', '9941dc54d4568b5c1cd1c9054d6c3463371eb782.tif', 'a6ef9669

## Data Preparation

In [ ]:
{{xxxxx}}

## Address Missing Values

In [ ]:
{{xxxxx}}

## Data Scoping Functions

In [ ]:
{{xxxxx}}

## Scope the Label and Text for Analysis

In [ ]:
{{xxxxx}}

# 3.&nbsp;Exploratory Data Analysis (EDA)

The EDA phase focuses on understanding the dataset, including data distribution and label counts. Various functions are used to inspect the structure of the dataset, visualize the label distribution, and assess the text length and word count of the documentation. The data is found to be somewhat imbalanced across categories.

## EDA Functions

In [ ]:
{{xxxxx}}

## EDA Analysis: Overview

In [ ]:
{{xxxxx}}

## EDA Analysis: Text Distribution

In [ ]:
{{xxxxx}}

## EDA Results

ADD HERE

# 4.&nbsp;Train-Validation-Test Split

Split the dataset into training, validation, and test sets. Use tratified splitting to ensure that the class distribution remains consistent across these sets. The distribution of records across the labels is visualized to ensure a balanced split.

## Test Split Functions

In [ ]:
{{xxxxx}}

## Test Split Analysis

In [ ]:
{{xxxxx}}

# 5.&nbsp;Data Cleansing & Text Normalization

{{xxxxx}}

## Core Normalization Functions

In [ ]:
{{xxxxx}}

## Apply Text Normalization

In [ ]:
{{xxxxx}}

# 6.&nbsp;Feature Engineering with TF-IDF

The TfidfVectorizer from scikit-learn is used to convert the text documents into numerical features. The vectorizer transforms the collection of documents into a matrix of token counts, which is then normalized using the Term Frequency-Inverse Document Frequency (TF-IDF) transformation. This matrix representation of the text data serves as input to the machine learning models.

## TF_IDF Functions

In [ ]:
{{xxxxx}}

## Vectorization

In [ ]:
{{xxxxx}}

# 7.&nbsp; Baseline Models: Supervised

## Model Functions

In [ ]:
{{xxxxx}}

## Build, Train, and Evaluate the Model

In [ ]:
{{xxxxx}}

# 8.&nbsp; Hyperparameter Tuning

## Tuning Functions

In [ ]:
{{xxxxx}}

## Execute Hyperparameter Tuning

In [ ]:
{{xxxxx}}

# 9.&nbsp;Final Prediction and Evaluation

## Evaluation Functions

In [ ]:
{{xxxxx}}

## Train the Final Model

In [ ]:
{{xxxxx}}

## Evaluate the Model

In [ ]:
{{xxxxx}}

## Explore Errors

In [ ]:
{{xxxxx}}

# 10.&nbsp;Scale the Auto-Classifier

## Auto-Classifier Functions

In [ ]:
{{xxxxx}}

## Rerun Process for L1

In [ ]:
{{xxxxx}}

## Rerun Process for L2

In [ ]:
{{xxxxx}}

# 11.&nbsp; Conclusions

{{xxxxx}}

## Results Summary

### Model Result Summary


**Baseline Results**

{{xxxxx}}

**Hyperparameter Tuning Results**

{{xxxxx}}

**Best Model Results**

{{xxxxx}}

**Best Model Performance**

{{xxxxx}}

## Model Comparison

### Model Comparisons and Findings

{{xxxxx}}

#### Baseline Results

{{xxxxx}}

#### Hyperparameter Tuning

{{xxxxx}}

#### Best Model Results

{{xxxxx}}

#### Performance Breakdown (Best Model)

{{xxxxx}}

#### Conclusion

{{xxxxx}}

## Concluding Observations

## Patterns and Conclusions Across the Models

{{xxxxx}}

# 12.&nbsp; References

**Kaggle Competition**

- [1] Will Cukierski. Kaggle. 2018. Histopathologic Cancer Detection. https://www.kaggle.com/c/histopathologic-cancer-detection/overview

**Kaggle File Import to Colab**
- [2] Abdul Qadir. 2020. Data Preparation Guide for detecting Histopathologic Cancer Detection https://towardsdatascience.com/data-preparation-guide-for-detecting-histopathologic-cancer-detection-7b96d6a12004/
- [3] Abdul Qadir. 2020. Histopathlogic-Cancer-Data-Preprocessing. https://github.com/DarthQadir/Histopathlogic-Cancer-Data-Preprocessing/tree/master

**References Resources**
- [4] Faramarz Kowsari. CNN_Histopathologic_Cancer_Detection_Colorado. 2023. https://www.kaggle.com/code/faramarzkowsari/cnn-histopathologic-cancer-detection-colorado
- [5] Kodai Fukuda. Histopathologic Cancer Detection with CNN. 2024. https://www.kaggle.com/code/kodaifukuda0311/histopathologic-cancer-detection-with-cnn